In [4]:
import tushare as ts
pro = ts.pro_api() 
#初始化
df1 = pro.index_dailybasic(start_date='20191008',end_date='20191017',ts_code='000001.SH',fields='ts_code,trade_date,total_mv,turnover_rate')
#大盘指数每日行情，目前主要是目前只提供上证综指，深证成指，上证50
#中证500，中小板指，创业板指的每日指标数据
df2 = pro.index_daily(ts_code='000001.SH',start_date='20191008',end_date='20191017')
#指数每日行情，用主要输出参数close,open,high,low,pre_close
#change涨跌点相对昨天收价，pct_chg涨跌幅相对昨天收盘价
#vol成交量手，amount成交额千元
df2['total_mv']=df1.total_mv
#将df1的total_mv合并df2
#dataframe增加一列用法，df['new']=[1,2,3,4] 
#dataframe增加一行用法，加loc,df.loc['new']=[1,2,3,4]
df2
#下一步连接数据库，再下一步，将其显示在网页上



,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount,total_mv
0,000001.SH,20191017,2977.3342,2979.9929,2986.7191,2969.5680,2978.7124,-1.3782,-0.0463,122527614.0,136051252.1,3.902802e+13
1,000001.SH,20191016,2978.7124,2992.6103,3010.4241,2975.9233,2991.0459,-12.3335,-0.4123,149885496.0,166791549.4,3.904096e+13
2,000001.SH,20191015,2991.0459,3005.6646,3005.6646,2986.3033,3007.8834,-16.8375,-0.5598,155376904.0,169531932.5,3.920118e+13
3,000001.SH,20191014,3007.8834,2993.9617,3026.3834,2989.8125,2973.6558,34.2276,1.1510,208614802.0,221201755.7,3.939485e+13
4,000001.SH,20191011,2973.6558,2954.8189,2980.7875,2943.0137,2947.7106,25.9452,0.8802,161203746.0,182747393.1,3.894539e+13
5,000001.SH,20191010,2947.7106,2923.7069,2949.2404,2918.2284,2924.8566,22.8540,0.7814,134239752.0,157159976.8,3.858208e+13
6,000001.SH,20191009,2924.8566,2902.0751,2924.8566,2891.5394,2913.5704,11.2862,0.3874,130424144.0,150940230.8,3.831022e+13
7,000001.SH,20191008,2913.5704,2905.7559,2933.0163,2905.7559,2905.1892,8.3812,0.2885,125535812.0,151980792.0,3.815001e+13
